#  Serve Falcon 7B model with Amazon SageMaker Hosting

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

---

In this example we walk through how to deploy and perform inference on the **Falcon 7B model** using the **Large Model Inference(LMI)** container provided by AWS using **DJL Serving** and **Accelerate**. The **Falcon 7B model** is a casual decoder model like the **Falcon 40B model**.


## Setup

Installs the dependencies required to package the model and run inferences using Amazon SageMaker. Update SageMaker, boto3 etc

In [ ]:
!pip install sagemaker boto3 --upgrade  --quiet
!pip install huggingface_hub

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from sagemaker.utils import name_from_base

## Imports and variables

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts

s3_code_prefix_accelerate = "baichuan-7B/code_baichuan7b"  # folder within bucket where code artifact will go

s3_model_prefix = (
    "baichuan-7B/model_baichuan7b"  # folder within bucket where code artifact will go
)
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

In [ ]:
## [OPTIONAL] Download the model from Hugging Face and upload the model artifacts on Amazon S3

In [20]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("./model")
local_model_path.mkdir(exist_ok=True)
model_name = "baichuan-inc/Baichuan-7B"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model", "*.py"]

# - Leverage the snapshot library to download the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:

# # define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")


Pretrained model will be uploaded to ---- > s3://sagemaker-cn-north-1-394224607677/baichuan-7B/model_baichuan7b/


In [22]:
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

Model uploaded to --- > s3://sagemaker-cn-north-1-394224607677/baichuan-7B/model_baichuan7b
We will set option.s3url=s3://sagemaker-cn-north-1-394224607677/baichuan-7B/model_baichuan7b


In [ ]:
# !rm -rf {model_download_path}

### 1. Create SageMaker compatible model artifacts

In order to prepare our model for deployment to a SageMaker Endpoint for hosting, we will need to prepare a few things for SageMaker and our container. We will use a local folder as the location of these files including **serving.properties** that defines parameters for the LMI container and **requirements.txt** to detail what dependies to install.

In [24]:
!rm -rf code_baichuan7b_accelerate
!mkdir -p code_baichuan7b_accelerate

In the **serving.properties** files  define the the **engine** to use and **model** to host. Note the **tensor_parallel_degree** parameter which is set to a value of **1** in this scenario. Since the entire model can fit on a sigle GPU we do not have to divide the model into multiple parts. In this case we will use a 'ml.g5.2xlarge' instance which provides **1** GPU. Be careful not to specify a value larger than the instance provides or your deployment will fail. 

In [25]:
%%writefile ./code_baichuan7b_accelerate/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = {{s3url}}

Writing ./code_baichuan7b_accelerate/serving.properties


In [26]:
%%writefile ./code_baichuan7b_accelerate/requirements.txt
einops
torch==2.0.1

Writing ./code_baichuan7b_accelerate/requirements.txt


In [27]:
## SKIP

# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("code_baichuan7b_accelerate/serving.properties").open().read())
Path("code_baichuan7b_accelerate/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize code_baichuan7b_accelerate/serving.properties | cat -n

     1	engine=Python
     2	option.tensor_parallel_degree=1
     3	option.s3url = s3://sagemaker-cn-north-1-394224607677/baichuan-7B/model_baichuan7b/


### 2. Create a model.py with custom inference code

SageMaker allows you to bring your own script for inference. Here we create our **model.py** file with the appropriate code for the Falcon 7B model.

In [28]:
%%writefile ./code_baichuan7b_accelerate/model.py
from djl_python import Input, Output
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from typing import Any, Dict, Tuple
import warnings
import logging

predictor = None


def get_model(properties):
        
    tensor_parallel_degree = properties["tensor_parallel_degree"]

    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    logging.info(f"Loading model in {model_location}")

    
    model = AutoModelForCausalLM.from_pretrained(
        model_location,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device_map="auto"
    )
    return generator


def handle(inputs: Input) -> None:
    global predictor
    if not predictor:
        predictor = get_model(inputs.get_properties())
    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None
    data = inputs.get_as_json()
    text = data["text"]
    text_length = data["text_length"]
    outputs = predictor(text, do_sample=True, min_length=text_length, max_length=text_length)
    result = {"outputs": outputs}
    return Output().add_as_json(result)

Writing ./code_baichuan7b_accelerate/model.py


### 3. Create the Tarball and then upload to S3 location
Next, we will package our artifacts as `*.tar.gz` files for uploading to S3 for SageMaker to use for deployment

In [29]:
!rm -f model.tar.gz
!rm -rf code_baichuan7b_accelerate/.ipynb_checkpoints
!tar czvf model.tar.gz -C code_baichuan7b_accelerate .

s3_code_artifact_accelerate = sess.upload_data("model.tar.gz", bucket, s3_code_prefix_accelerate)
print(f"S3 Code or Model tar for accelerate uploaded to --- > {s3_code_artifact_accelerate}")

./
./requirements.txt
./serving.properties
./model.py
S3 Code or Model tar for accelerate uploaded to --- > s3://sagemaker-cn-north-1-394224607677/baichuan-7B/code_baichuan7b/model.tar.gz


### 4. Define a serving container, SageMaker Model and SageMaker endpoint
Now that we have uploaded the model artifacts to S3, we can create a SageMaker endpoint.


#### Define the serving container
Here we define the container to use for the model for inference. We will be using SageMaker's Large Model Inference(LMI) container using Accelerate. 

In [30]:
# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
inference_image_uri = (
    f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.22.1-deepspeed0.8.3-cu118"
)
print(f"Image going to be used is ---- > {inference_image_uri}")


Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.22.1-deepspeed0.8.3-cu118


#### Create SageMaker model, endpoint configuration and endpoint.


In [31]:
model_name_acc = name_from_base(f"baichuan7b-model-acc")
print(model_name_acc)

baichuan7b-model-acc-2023-08-04-09-28-49-360


In [32]:
s3_code_artifact_accelerate="s3://sagemaker-cn-north-1-394224607677/baichuan-7B/code_baichuan7b/model.tar.gz"
create_model_response = sm_client.create_model(
    ModelName=model_name_acc,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact_accelerate},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

Created Model: arn:aws-cn:sagemaker:cn-north-1:394224607677:model/baichuan7b-model-acc-2023-08-04-09-28-49-360


In [33]:
model_name = model_name_acc
print(f"Building EndpointConfig and Endpoint for: {model_name}")

Building EndpointConfig and Endpoint for: baichuan7b-model-acc-2023-08-04-09-28-49-360


In [34]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            #"ModelDataDownloadTimeoutInSeconds": 600,
            #"ContainerStartupHealthCheckTimeoutInSeconds": 600,
            # "VolumeSizeInGB": 512
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws-cn:sagemaker:cn-north-1:394224607677:endpoint-config/baichuan7b-model-acc-2023-08-04-09-28-49-360-config',
 'ResponseMetadata': {'RequestId': '5d738943-1909-4d35-bebb-89fc62f92391',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5d738943-1909-4d35-bebb-89fc62f92391',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '136',
   'date': 'Fri, 04 Aug 2023 09:28:56 GMT'},
  'RetryAttempts': 0}}

In [35]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws-cn:sagemaker:cn-north-1:394224607677:endpoint/baichuan7b-model-acc-2023-08-04-09-28-49-360-endpoint


In [36]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws-cn:sagemaker:cn-north-1:394224607677:endpoint/baichuan7b-model-acc-2023-08-04-09-28-49-360-endpoint
Status: InService


### Run Inference

In [37]:
%%time

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"text": "北京在哪里?", "text_length": 150}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 12.6 s


'{\n  "outputs":[\n    {\n      "generated_text":"北京在哪里?\\n北京位于华北平原北部,背靠燕山,面向渤海,是我国的首都、直辖市和国家中心城市,其市中心是北京老城区。北京西、北、东3面环山,东与天津相邻,而南部则是渤海湾,北京的地理位置北纬39度54分,东经116度23分。其北、西、东都与各省市的省会城市隔海相望,南面是珠江海口附近。\\n北京也是近代中国历史的象征和标志,因为北京是我国近代史和当代史最浓墨重彩的一笔。\\n\\n【北京地理"\n    }\n  ]\n}'

In [ ]:
%%time


response_model = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps({"text": "北京市在哪里?", "text_length": 150}),
        ContentType="application/json",
    )

response_model["Body"].read().decode("utf8")

In [ ]:
start_time = time.time()

while (time.time() - start_time) < 300:  # 300 seconds = 5 minutes
    response_model = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps({"text": "故宫是哪个朝代创建的?", "text_length": 150}),
        ContentType="application/json",
    )

    print("Loop restarting - answer: " + response_model["Body"].read().decode("utf8"))

### Clean Up

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-7b-accelerate.ipynb)
